基本上可以跳過不用跑

除非要重train bert model

In [8]:
import torch , json, glob
from pytorch_pretrained_bert import *
import logging
logging.basicConfig(level=logging.INFO)
import os,sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random
from multiprocessing import Pool, Manager

In [3]:
samples_pid_train = pickle.load(open('./data/tree-rep-profiles-partial/TRAIN_pidNames.pkl','rb'))
samples_pid_valid = pickle.load(open('./data/tree-rep-profiles-partial/DEV_pidNames.pkl','rb'))
samples_pid_test = pickle.load(open('./data/tree-rep-profiles-partial/TEST_pidNames.pkl','rb'))

In [3]:
# all many2many pid
all_m2m_pid = []
root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            all_m2m_pid.extend(hl_list)
all_m2m_pid = set(all_m2m_pid)

100%|██████████| 183/183 [00:00<00:00, 3255.20it/s]


In [4]:
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
# api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

In [5]:
corpus = []
dil= r"[@#]+\ *" #等號、底線被保留

root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl in hl_list:
                try:
                    if hl in samples_pid_test: #移除在test裡面的hkl
                        hl_list.remove(hl)
                    if hl in samples_pid_valid: #移除在valid裡面的hkl
                        hl_list.remove(hl)
                except ValueError:
                    pass
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
#             hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for profile in hl_list:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line) #lower小寫?
                    temp = temp.replace('PR',' ')
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    if temp.split(' ')[0] not in api_li:
                        print('=o2m_ERR:=',profile,'=>',temp)
                        continue
                    corpus.append(temp)
                    corpus.append('\n')
                corpus.append('\n')

root_dir = './data/tree-rep-profiles_one2one/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl in hl_list:
                try:
                    if hl in all_m2m_pid: #移除已經有的pid
                        hl_list.remove(hl)
                except ValueError:
                    pass
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for profile in hl_list:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line) #小寫?
                    temp = temp.replace('PR','')
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    if temp.split(' ')[0] not in api_li:
                        print('=o2o_ERR:=',profile,'=>',temp)
                        continue
                    corpus.append(temp)
                    corpus.append('\n')
                corpus.append('\n')

with open('data/bert_corpus/o2o_o2m_Bert_0530.txt','w') as f:
    f.write(''.join(corpus))
corpus[-100:]

100%|██████████| 166/166 [00:22<00:00, 10.88it/s]


['\n',
 'DeleteFile SYS EXE Ret 0',
 '\n',
 'CreateFile USR EXE GENERIC_READ OPEN_ALWAYS FILE_SHARE_READ;FILE_SHARE_WRITE Ret 0',
 '\n',
 'CreateFile SYS EXE GENERIC_WRITE OPEN_ALWAYS FILE_SHARE_READ;FILE_SHARE_WRITE Ret 0',
 '\n',
 'RegCreateKey HKCU hkey_current_user Software\\VB and VBA Program Settings\\Explorer\\Process Ret 0',
 '\n',
 'RegSetValue HKCU software\\vb and vba program settings\\explorer\\process\\lo REG_SZ 1 Ret 0',
 '\n',
 'CreateProcess CMD Ret 0',
 '\n',
 'LoadLibrary SYS version DLL Ret 0',
 '\n',
 'LoadLibrary SYS explorer EXE Ret 0',
 '\n',
 'CreateFile SYS EXE GENERIC_READ OPEN_EXISTING FILE_SHARE_DELETE;FILE_SHARE_READ Ret 0',
 '\n',
 'LoadLibrary SYS explorer EXE Ret 0',
 '\n',
 'CreateFile SYS EXE GENERIC_READ OPEN_EXISTING FILE_SHARE_DELETE;FILE_SHARE_READ Ret 0',
 '\n',
 'RegQueryValue HKCU soft_ms_win_explorer_shellFolders\\* SUBK cache 0 12e8f4 Ret 0',
 '\n',
 'LoadLibrary SYS advapi32 DLL Ret 0',
 '\n',
 'LoadLibrary SYS oleaut32 DLL Ret 0',
 '\n',
 'R

In [6]:
with open('data/bert_corpus/o2o_o2m_Bert_0530.txt','r') as f: #asp1_sentences_rep_0116
    corpus = f.read().splitlines()
model = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model, do_lower_case=False)
lens = []
for line in tqdm(corpus):
    tokens = tokenizer.tokenize(line)
    length = len(tokens)
    if length < 2:
        continue
    else:
        lens.append(length)
print(max(lens),min(lens))

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/leoqaz12/.cache/torch/pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
100%|██████████| 4666391/4666391 [32:57<00:00, 2359.95it/s] 

197 5


In [10]:
lens_sort = sorted(lens)
for i in [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10]:
    idx = lens.index(lens_sort[-1])
    print(max(lens[idx-1] + lens[idx] , lens[idx+1] + lens[idx]))

239
239
239
239
239
239
239
239
239
239


python3 pregenerate_training_data.py --train_corpus ../../../data/bert_corpus/o2o_o2m_Bert_0530 --bert_model bert-base-multilingual-cased --output_dir ../../../model/bert_pretrained/less_pretrained_bert/ --epochs_to_generate 3 --max_seq_len 239

python3 finetune_on_pregenerated.py --pregenerated_data ../../../model/bert_pretrained/less_pretrained_bert/ --bert_model bert-base-multilingual-cased --output_dir /SAMBA/r06725035/model/less_pretrain_bert/bert/  --epochs 3 --gradient_accumulation_steps 3 --learning_rate 2e-5  --train_batch_size 48

In [6]:
dil= r"[@#]+\ *" #等號、底線被保留
def clean_profile(profile_path,dil=dil):
    '''
    input: profile (path string)
    output: clean profile (corresponding path)
    '''
    corpus = []
    with open(profile_path,encoding='ISO 8859-1') as f:
        lines = f.read()
    lines = re.sub(r'[^\x00-\x7F]+','', lines)
    lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
    lines = lines.splitlines()
    for line in lines:
        temp = re.sub(dil," ",line) #小寫?
        temp = temp.replace('PR','')
        temp = temp.split(" ")
        temp = list(filter(None, temp))
        temp = ' '.join(temp)
        corpus.append(temp)
        corpus.append('\n')
    with open(profile_path.replace('.profile','.profile_clean'),'w') as f:
        f.write(''.join(corpus))

In [3]:
%%capture
def store_feature(profile_clean,model_path,out,max_seq_length=239,batch_size=32):
    '''
    input: profile_clean (path string)
    output: profile feature json dicts
    '''
#     out = profile_clean.replace('.profile_clean','.feature')
    !python3 ./pytorch-pretrained-BERT/examples/extract_features.py --input_file $profile_clean --output_file $out --bert_model $model_path --batch_size $batch_size #&>/tmp/tmp

In [2]:
model_path1 = '/DATA/r06725035/model/less_pretrain_bert/bert/pytorch_model.bin'
model_state_dict1 = torch.load(model_path1,  map_location='cpu')
model1 = BertModel.from_pretrained(pretrained_model_name_or_path='bert-base-multilingual-cased',state_dict=model_state_dict1)
model1 = torch.nn.DataParallel(model1)
model1.eval()

model_path2 = './model/bert_pretrained/bert/pytorch_model.bin'
model_state_dict2 = torch.load(model_path2,  map_location='cpu')
model2 = BertModel.from_pretrained(pretrained_model_name_or_path='bert-base-multilingual-cased',state_dict=model_state_dict2)
model2 = torch.nn.DataParallel(model2)
model2.eval()
# model = BertModel.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='bert-base-multilingual-cased', do_lower_case=False)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/leoqaz12/.cache/torch/pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/leoqaz12/.cache/torch/pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpd1nf5u0i
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_

In [3]:
def get_feature(hl_path,model1=model1,model2=model2,tokenizer=tokenizer,sent=True):
    last = [-1,-2,-3,-4] #要拿最後幾層?
    cls_np_all = []
    token_np_all = []
    with open(hl_path,encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    for line in lines:
        line = '[CLS] ' + line + ' [SEP]'
        tokenized_text = tokenizer.tokenize(line)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [0]*len(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
#         tokens_tensor = tokens_tensor.cuda() #.to(device)
#         segments_tensors = segments_tensors.cuda() #.to(device)
        if sent:
            model1.cuda()
            tokens_tensor = tokens_tensor.cuda()
            segments_tensors = segments_tensors.cuda()
        else:
            model2.cuda()
            tokens_tensor = tokens_tensor.cuda() #.to(device)
            segments_tensors = segments_tensors.cuda() #.to(device)
#         print(len(tokenized_text))
        if sent: #只拿[CLS] 最後四層
            with torch.no_grad():
                encoded_layers, _ = model1(tokens_tensor, segments_tensors)
            cls_np = []
            for layer in last:
                cls_np.append(np.array(encoded_layers[layer].cpu())[0][0])
            last_np = np.sum(cls_np,axis=0)
            cls_np_all.append(last_np)
        else:
            with torch.no_grad():
                encoded_layers, _ = model2(tokens_tensor, segments_tensors)
            length = len(encoded_layers[-1][0])
            all_np = []
            for layer in last:
                for token in range(1,length-1):
                    array = np.array(encoded_layers[layer].cpu())[0][token]
                    all_np.append(array)
            last_np = np.average(all_np,axis=0)
            token_np_all.append(last_np) #只有一個lline!!
    try:
        del model1
        del model2
    except:
        torch.cuda.empty_cache()
    if sent:
        return np.array(cls_np_all) #[CLS]
    else:
        return np.array(token_np_all) # tokens
            
#     break

In [83]:
# layers = [-1,-2,-3,-4]
# bert_model = 'bert-base-multilingual-cased'
# load_model = '/DATA/r06725035/model/less_pretrain_bert/bert/pytorch_model.bin'
# do_lower_case = False
# input_file = 'test.txt'
# max_seq_length = 239
# batch_size = 32

In [4]:
root_dir = './data/tree-rep-profiles_one2many/normal/'


In [ ]:
#[CLS]
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for hl in hl_list:
            clean_profile(hl)
#         in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile_clean"), hl_list))
        for hl in hl_list:
            cls_feature = get_feature(hl,model1,model2,sent=True) #還傳該hkl的兩種feature
            cls_file = hl.replace('.profile_clean','_clsfeature.pkl')
#             token_file = hl.replace('.profile_clean','_tokenfeature.pkl')
            pickle.dump(file=open(cls_file,'wb'),obj=cls_feature)
#             pickle.dump(file=open(token_file,'wb'),obj=tokens_feature)
            

 67%|██████▋   | 123/184 [6:20:48<4:00:37, 236.67s/it] 

# Copy corresponding feature
* src= './data/tree-rep-profiles_one2many/normal/'
* dest = './data/tree-rep-profiles-partial/normal/'
    * './data/tree-rep-profiles-partial/DEV/'
    * './data/tree-rep-profiles-partial/TEST/'
    
- 需要先將src dir也進行合併以後才可以使用下列code

In [25]:
# src_dir = './data/tree-rep-profiles_one2many/normal/'

In [24]:
# def restore_feature(root_dir,src_dir=src_dir):
#     fam_dir = next(os.walk(root_dir))[1]
#     for fam in tqdm(fam_dir):
#         tree_dir = next(os.walk(root_dir + fam))[1]
#         for tree in tree_dir:
#             in_directory = root_dir + fam +  '/' + tree + '/'
#             hl_list = next(os.walk(in_directory))[2]
#             hl_list = [os.path.join(in_directory, f) for f in hl_list]
#             hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
#             for hl in hl_list:
#                 files = glob.glob(src_dir + fam + '/' + tree + '/'+hl.split('/')[-1].replace('.profile','*'))
#             for file in files:
#                 shutil.copy(file,in_directory)

In [26]:
# root_dir = './data/tree-rep-profiles-partial/normal/'
# restore_feature(root_dir)
# root_dir = './data/tree-rep-profiles-partial/DEV/'
# restore_feature(root_dir)
# root_dir = './data/tree-rep-profiles-partial/TEST/'
# restore_feature(root_dir)

100%|██████████| 74/74 [00:00<00:00, 158.25it/s]
